# QGIS
## 安装
- [ubuntu安装QGIS（一）](https://blog.csdn.net/u014124220/article/details/51900854)

**pyqt安装**

Ancona安装pyqt，在新的conda 版本中自带的是 qt5，想要安装其他的版本可以使用如下命令安装，如下：
```bash
# 安装最新版本
conda install pyqt

# 安装指定版本
conda install pyqt=4
conda install pyqt=4.11
```

**qgis 插件安装**
1. plugin builder: 用于构建QGIS插件的基本结构

使用 plugin builder 首次构建的插件，需要在插件的根目录中私用 make 进行编译一次，否则在重启 QGIS 重启的时候会发生错误。

2. plugin reloader: 此插件是一个实验性质的插件，可以不用重启QGIS的情况下重新加载插件，需要在插件管理里在Settings中将 `Show also experimental plugins` 选项打开。

3. OpenLayer Plugin: 此插件安装完成之后需要在 `web` 菜单中查看。

## 插件开发

### 流程

开发好的插件放在指定的目录，重新启动QGIS即可加载到插件。

在Unix系统中，插件存放的目录在 `~/.qgis2/python/plugins`; 在Mac系统中，存放的目录在 `(qgis_prefix)/share/qgis/python/plugin`

开发需要以下几个步骤：
1. 明确需要做的事情，若功能其他插件已有实现可以使用现有插件
2. 创建必要的文件。首先，创建初始化文件__init__.py文件，用来设置插件加载前的某些属性值。其次，创建插件主体文件plugin.py，你所有的插件操作将会在此文件中完成。最后，用QT-Designer设计插件界面，生成界面文件form.ui，伴随ui文件一般会有一份资源文件生成，即resources.qrc。
3. 在plugin.py中填写必要的代码。
4. 测试。重启QGIS看能不能将插件加载进来并顺利运行。
5. 发布插件，供别人使用

### 所需文件说明

QGIS 插件开发需要的主要文件如下：
1. `__init__.py` : 
2. `plugin.py` : 插件主体，所有的插件操作都必须在这里完成，也是写代码最多的部分；
3. `metadata.txt` : QGIS>=1.8.0版本要求提供，是插件元数据，描述了插件的基本信息，如版本号，插件名和其他一些插件网址，框架信息等。在`__init__.py`文件中会用到，用来获取上述插件相关信息。然而从QGIS2.0开始，所有的插件元数据信息只能在metadata.txt中设置了，在`__init__.py`中的设置将被视为无效。
4. `resources.qrc` : QT-Designer创建的XML文档，包含了界面资源的相对路径
5. `resources.py`
6. `form.ui` : QT-Designer创建的界面文件
7. `form.py` : 由 form.ui 转换而来

####  \_\_init\_\_.py
有如下信息：
```python
def name(): 
      return "Zoom plugin" 
def description():
      return "Zooms to a point when the user hits the button."
# 插件版本
def version(): 
      return "Version 0.1"
def authorName():
        return "XXXX"
# 需要的QGIS版本，在低于此版本的QGIS中安装无效
def qgisMinimumVersion():
      return "2.18"
def category():
        return "Raster"
# 加载插件主文件
def classFactory(iface): 
      # load Zoomer class from file Zoomer
      from Zoomer import Zoomer 
      return Zoomer(iface)
```

在1.9.90版本后，在__init__.py中新增了函数“category()”使得插件在菜单栏Raster、Vector、Database和Web下都可以了。该函数的作用就是定义插件的显示菜单，但目前其值只能选“Vector”、“Raster”、“Database”、“Web”和“Layers”中的一个

#### metadata.txt

在1.8版本之后，你必须添加该文件以描述你的插件信息

```
; the next section is mandatory
[general]
# 在插件管理中心显示的名字
name=Vehicle Lane 
qgisMinimumVersion=2.18
description=Zooms to a point when the user hits the button.
category=Raster
version=version 0.1
; end of mandatory metadata
```

#### plugins.py

在该类中定义插件操作。值得提醒的是该类中的classFactory()函数。该函数会在QGIS加载外部插件时调用，用来接收QgisInterface类实例的引用和（必须）返回自定义插件类的实例，即你的插件实例。
```python
class Zoomer: 

      def __init__(self, iface):
        # Save reference to the QGIS interface
        self.iface = iface
        ...
```

## QGIS中相关方法

### 图层
**获取图层的方法**
1. `iface.legendInterface().layers()`
2. `iface.mapCanvas().layers()`
3. `layers = QgsMapLayerRegistry.instance().mapLayers()`

获取被激活的图层（即被选中的图层）：`iface.activeLayer()`

**获取图层的字段及其值**

`layer.pendingFileds()` `layer.getFeatures()`

获取向量图的坐标(经纬度)，首先获取图层的特征，之后通过 `field.geometry()` 该方法返回一个几何对象，通过`asPoint()`方法获取点的`x`、`y`坐标。若是线或者多边形则使用`asPolyline()`、`asPolygon()`。

`geometry()` 返回的对象包含以下的方法：
```json
['addPart', 'addPartGeometry', 'addRing', 'adjacentVertices', 'angleAtVertex', 'area', 'asGeometryCollection', 'asMultiPoint', 'asMultiPolygon', 'asMultiPolyline', 'asPoint', 'asPolygon', 'asPolyline', 'asQPointF', 'asQPolygonF', 'asWkb', 'avoidIntersections', 'boundingBox', 'buffer', 'centroid', 'closestSegmentWithContext', 'closestVertex', 'closestVertexWithContext', 'combine', 'compare', 'contains', 'convertPointList', 'convertToMultiType', 'convertToSingleType', 'convertToStraightSegment', 'convertToType', 'convexHull', 'createGeometryEngine', 'createPolygonFromQPolygonF', 'createPolylineFromQPolygonF', 'crosses', 'deletePart', 'deleteRing', 'deleteVertex', 'difference', 'disjoint', 'distance', 'distanceToVertex', 'draw', 'equals', 'exportToGeoJSON', 'exportToWkt', 'extrude', 'fromMultiPoint', 'fromMultiPolygon', 'fromMultiPolyline', 'fromPoint', 'fromPolygon', 'fromPolyline', 'fromQPointF', 'fromQPolygonF', 'fromRect', 'fromWkb', 'fromWkt', 'geometry', 'insertVertex', 'interpolate', 'interpolateAngle', 'intersection', 'intersects', 'isEmpty', 'isGeosEmpty', 'isGeosEqual', 'isGeosValid', 'isMultipart', 'length', 'lineLocatePoint', 'makeDifference', 'mapToPixel', 'mergeLines', 'moveVertex', 'nearestPoint', 'offsetCurve', 'overlaps', 'pointOnSurface', 'requiresConversionToStraightSegments', 'reshapeGeometry', 'rotate', 'setGeometry', 'shortestLine', 'simplify', 'smooth', 'smoothLine', 'smoothPolygon', 'splitGeometry', 'sqrDistToVertexAt', 'symDifference', 'touches', 'transform', 'translate', 'type', 'unaryUnion', 'validateGeometry', 'vertexAt', 'vertexIdFromVertexNr', 'vertexNrFromVertexId', 'within', 'wkbSize', 'wkbType']
```

In [ ]:
layer = iface.activeLayer()
for i, f in enumerate(layer.getFeatures()):
    geo = f.geometry()
    # 线
    if geo.type() == 1:
        print(geo.asPolyline())
    # 点
    elif geo.type() == 0:
        print(geo.asPoint(), geo.asPoint().x())
    if i == 10:
        break
# 输出
# ((75.9571,30.8504), 75.95707224036518)
# ((75.9331,17.6254), 75.93305977107549)      
        
        


## UI

PyQt4 中 QMessageBox 在 QtGui中，而 PyQt5 在 QWigets中。

### QFileDialog 文件选取对话框
用于文件的选择，点击按钮弹出一个文件选取对话框，返回一个被选中的文件的全路径。 `QFileDialog.getSaveFileName(self.dlg, 'Select output file', '', '*.txt')`

- parent：上级窗口
- caption：弹出框的标题
- directory：文件夹路径
- filter：文件过滤器


### lineEdit 文本编辑框
通过组件的名字来操作组件，常见方法有
- `clear()`：清空文本框中的值
- `setText()`：为文本框设值
- `text()`：获取文本框中的值

### 按钮
#### pushButton 按钮
也是通过组件名进行相关操作，如为按钮绑定事件 `pushButton.clicked.connect(self.select_output_file)`
```python
def select_output_file(self):
    filename = QFileDialog.getSaveFileName(self.dlg, 'Select output file', '', '*.txt')
    self.dlg.lineEdit.setText(filename)
```

### 下来框  comboBox
此组件含有如下方法
- `addItems`：向 comboBox中添加元素
- `currentIndex()`：获取当前comboBox被选中值的索引


## QGIS 中的一些概念

UTM (国际横麦卡托)投影

UTM Zone 54N，QGIS --> CRS(Coordinate Reference System)

属性表格，属性操作（open attributes table）

文字图层（使用特殊字符分割的数据，如csv数据）

OSM： open street map

**计算时需要注意：**使用合理的坐标系统很重要

植生指标（NDVI）

DEM (地形模型 Terrain Models)：Digital Elevation Matrix，数字高程矩阵

WMS (Web Map Service,线上地图服务)

空间对位（Georeferencing）

## 问题

- 距离矩阵运算时，若空间中的点有问题，距离矩阵运算时会发生错误，这是可以使用 `Vector -> Geometry Tools -> Check Validity` 检查处有问题的点，在正确的电商作分析。

[AttributeError: 'NoneType' object has no attribute 'asPoint'](https://gis.stackexchange.com/questions/263904/attributeerror-nonetype-object-has-no-attribute-aspoint/264525)

## 参考文章
- [QGIS Python程式设计的初步上手](http://www.qgistutorials.com/zh_TW/docs/getting_started_with_pyqgis.html)
- [在QGIS下开发python插件](https://blog.csdn.net/xiluoduyu/article/details/9992179)
- [以 Python 製作附加元件](http://www.qgistutorials.com/zh_TW/docs/building_a_python_plugin.html) & [Building a Python Plugin](https://www.qgistutorials.com/en/docs/building_a_python_plugin.html)
- [使用自订的Python表达式函数](http://www.qgistutorials.com/zh_TW/docs/custom_python_functions.html)
